##### Read data

In [16]:
import math

In [17]:
with open('SVO_timetable.csv') as f:
    content = f.readlines()
content = [x.split(";") for x in content]
content = content[1:]
timetable = []
data = []
for elem in content:
    timetable.append(elem[0])
for i in range(108):
    data.append(int(content[i][1]))

##### First route: airplanes -> temporary storage

This function reshuffles carriers inside the first route. If at monent i there isn't enough carriers
at slot i, it adds new ones. Otherwise it adss extra carriers from slot i to slot i+1. Irrespective
of the amount of carriers at moment i it adds all carriers to the slot i+4 (in 20 minutes all carriers going to the temporary storage will come back) and adds all containres to the array at slot i+20 (in maximum 1:40 minutes these containers should be taken from the temporary storage to be delivered to terminal in maximum 2 hours).

Constants: 108(+4) - amount of slots from 9:00 to 17:55 + 4(to fulfill the condition of returning carriers to airplanes); 128(+4)- amount of slots 9:00 to 17:55 + 2 hours (the last containers deliverd at 17:55 have to be taken maximum at 19:35).  

In [18]:
def first_route(content):
    first = [[] for i in range(1, 113)]
    Q1 = 0
    weights = [0 for i in range(1, 131)]
    for i in range(len(content)):
        weights[i+20] = content[i]
        size = math.ceil(content[i]/4)
        if size > len(first[i]):
            temp = Q1
            Q1 += size-len(first[i])
            for j in range(temp+1, temp+size-len(first[i])+1):
                first[i].append(j)
        else:
            while len(first[i]) > size:
                extra = first[i][-1]
                first[i].pop()
                first[i+1].append(extra)
        for k in first[i]:
            first[i+4].append(k)
    return Q1, first, weights

In [19]:
Q1 = first_route(data)[0]
first = first_route(data)[1]
weights = first_route(data)[2]

##### Amount of airplnes at route airplanes -> temporary storage

In [20]:
print(Q1)

15


##### Carriers' movement map at route airplanes -> temporary storage

In [21]:
with open("SVO_first.txt", 'w') as f:
    for i in range(108):
        f.write(timetable[i] +" %s\n" % first[i])

##### Optimization of containers' amount spread over the time grid

To minimize the amount of carriers the containers can be rebalanced only to slots were amount of containers is not divisible by 4. From 10:40 (slot 21) to 19:35 (slot 128) containers can be rebalanced and 20 is the starting point of weights distribution.

In [22]:
def weights_rebalance(weights):
    for i in range(20, 128):
        for j in range(max(20,i-18), i):
            if weights[j] % 4 != 0:
                if weights[j] < 4 and weights[i]-(4-weights[j]) >= 0 and weights[i]-(4-weights[j]) % 4 != 0 :
                    weights[i] -= 4-weights[j]
                    weights[j] += 4-weights[j]
                elif weights[j] < 8 and weights[i]-(8-weights[j]) >= 0 and weights[i]-(8-weights[j]) % 4 != 0 :
                    weights[i] -= 8-weights[j]
                    weights[j] += 8-weights[j]
                elif weights[j] < 12 and weights[i]-(12-weights[j]) >= 0 and weights[i]-(12-weights[j]) % 4 != 0 :
                    weights[i] -= 12-weights[j]
                    weights[j] += 12-weights[j]
                elif weights[j] < 16 and weights[i]-(16-weights[j]) >= 0 and weights[i]-(16-weights[j]) % 4 != 0 :
                    weights[i] -= 16-weights[j]
                    weights[j] += 16-weights[j]
    return weights

In [23]:
weights = weights_rebalance(weights)

##### Distribution of containers' amount taken from the temporary storage spread over the time grid

In [24]:
with open("SVO_weights.txt", 'w') as f:
    for i in range(len(weights)):
        f.write(timetable[i] +" %s\n" % weights[i])

##### Carriers' transfer from route 1 to route 2 

Transfer consists of two parts. In the first part carriers which arrive at the temporary storage from 5:50 p.m. to 6:05 p.m. will not have time to come back, so from this point they are fully availiable at route 2. All other carries are also transfered from the planes to temprorary storage to route 2 (the condition that they will not be in need is checked upside down).

In [25]:
def transfer(first):
    second = [[] for i in range(1, 133)]
    temp = set()
    for i in range(104, 108):
        for elem in first[i]:
            second[i+2].append(elem)
            temp.add(elem)
    while len(temp) < Q1:
        for j in range(103, 0, -1):
            for k in range(len(first[j])):
                if first[j][k] not in temp:
                    second[j+2].append(first[j][k])
                    temp.add(first[j][k])
    return second

In [26]:
second = transfer(first)

##### Routes: temporary storage -> terminal

The logic of carriers' distribution at route 2 is similar to route 1.

In [27]:
def second_route(Q1, weights, second):
    Q2 = Q1
    for i in range(20, 128):
        size = math.ceil(weights[i]/4)
        if size > len(second[i]):
            temp = Q2
            Q2 += size-len(second[i])
            for j in range(temp+1, temp+size-len(second[i])+1):
                second[i].append(j)
        else:
            while len(second[i]) > size:
                temp1 = second[i][-1]
                second[i].pop()
                second[i+1].append(temp1)
        if i < 124: 
            for k in second[i]:
                second[i+8].append(k)
    return second, Q2
    

In [28]:
Q2 = second_route(Q1, weights, second)[1]
second = second_route(Q1, weights, second)[0]

##### Necessary carriers' amount at two routes

In [29]:
print(Q2)

38


##### Carriers' movement map at route temporary storage -> terminal

In [30]:
with open("SVO_second.txt", 'w') as f:
    for i in range(128):
        f.write(timetable[i] +" %s\n" % second[i])